In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## 1) 載入資料集

In [5]:
df_train = pd.read_csv('/kaggle/input/aia-st3-forest-type-prediction-dl/train.csv')
df_train = df_train.drop(labels=['Id'],axis=1) # 移除Id

FileNotFoundError: [Errno 2] File /kaggle/input/aia-st3-forest-type-prediction-dl/train.csv does not exist: '/kaggle/input/aia-st3-forest-type-prediction-dl/train.csv'

## 2) 檢查缺失值
使用 numpy 所提供的函式來檢查是否有 NA 缺失值，假設有缺失值使用dropna()來移除。使用的時機在於當只有少量的缺失值適用，若遇到有大量缺失值的情況，或是本身的資料量就很少的情況下建議可以透過機器學習的方法補值來預測缺失值。

```python
# 移除缺失值
train=train.dropna()
```

In [ ]:
# checked missing data
print("Before data clean(NAN mount):",len(np.where(np.isnan(df_train))[0]))

## 3) 資料前處理

#### **特徵標準化**
通常有兩種標準化的方法：
- min max normalization：
    - 會將特徵數據按比例縮放到0到1的區間，（或是-1到1）。
- standard deviation normalization：
    - 會將所有特徵數據縮放成平均為0、平方差為1。
    
#### **特徵組合**
特徵需要適當地增加和減少，以提升精確度並減少計算時間。
- 增加特徵：特徵組合 (Feature Combination)、群聚編碼 (GroupBy Encoding)、產生合成樣本(Oversampling)
- 減少特徵：特徵篩選(Feature Selection)、剔除一些樣本(Undersampling)

In [6]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

def data_preprocessing(df_input):
    # numeric feature standardization
    sc = StandardScaler()    
#     sc = MinMaxScaler()
    df=sc.fit_transform(df_input.iloc[:,0:54])
    return df

In [7]:
X = data_preprocessing(df_train)

NameError: name 'df_train' is not defined

In [8]:
X.shape

NameError: name 'X' is not defined

## 3) One hot encoding
對`Cover Type`輸出欄位的資料做 one-hot encoding，使用 Keras 提供的工具函式 to_categorical 將每筆資料的輸出值 y 轉換成一個向量。

In [8]:
y = df_train['Cover_Type'].values-1
y=y.reshape((-1, 1))

## 4) 切割訓練集與測試集

In [2]:
from sklearn.model_selection import train_test_split
X_train , X_test , y_train , y_test = train_test_split(X,y , test_size=.3 , random_state=42)

NameError: name 'X' is not defined

In [10]:
print('訓練資料: ',X_train.shape)
print('測試資料: ',X_test.shape)

訓練資料:  (10054, 54)
測試資料:  (4310, 54)


## XGBoost 模型

In [1]:
from xgboost import XGBClassifier

# 建立XGBClassifier模型
xgboostModel = XGBClassifier(n_estimators=1000, learning_rate= 0.3)
# 使用訓練資料訓練模型
xgboostModel.fit(X_train, y_train)
# 使用訓練資料預測分類
predicted = xgboostModel.predict(X_train)

NameError: name 'X_train' is not defined

In [ ]:
from sklearn.metrics import accuracy_score

predicted = xgboostModel.predict(X_train)
print('訓練集準確率: ',accuracy_score(y_train, predicted))
predicted = xgboostModel.predict(X_test)
print('測試集準確率:',accuracy_score(y_test, predicted))

## 5) 建立網路模型

In [11]:
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import StackingClassifier

estimators = [
    ('rf', RandomForestClassifier()),
    ('lr', LogisticRegression()),
    ('svc', svm.SVC()),
    ('knn', KNeighborsClassifier()),
    ('dt', DecisionTreeClassifier())
]
clf = StackingClassifier(
    estimators=estimators, final_estimator= XGBClassifier()
)

clf.fit(X_train, y_train).score(X_test, y_test)


0.8587006960556844

## 測試集預測

In [12]:
from sklearn.metrics import accuracy_score

predicted = clf.predict(X_train)
print('訓練集準確率: ',accuracy_score(y_train, predicted))
predicted = clf.predict(X_test)
print('測試集準確率:',accuracy_score(y_test, predicted))

訓練集準確率:  0.9999005370996619
測試集準確率: 0.8587006960556844
